In [25]:
import pandas as pd
import requests
import time
import os 
from requests.exceptions import RequestException
from dateutil import parser

In [18]:
df = pd.read_csv('../data/raw/books_1.Best_Books_Ever.csv')
df.head()

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9999999999999,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,9999999999999,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52478 entries, 0 to 52477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            52478 non-null  object 
 1   title             52478 non-null  object 
 2   series            23470 non-null  object 
 3   author            52478 non-null  object 
 4   rating            52478 non-null  float64
 5   description       51140 non-null  object 
 6   language          48672 non-null  object 
 7   isbn              52478 non-null  object 
 8   genres            52478 non-null  object 
 9   characters        52478 non-null  object 
 10  bookFormat        51005 non-null  object 
 11  edition           4955 non-null   object 
 12  pages             50131 non-null  object 
 13  publisher         48782 non-null  object 
 14  publishDate       51598 non-null  object 
 15  firstPublishDate  31152 non-null  object 
 16  awards            52478 non-null  object

In [20]:
df['publishDate'].value_counts

<bound method IndexOpsMixin.value_counts of 0                  09/14/08
1                  09/28/04
2                  05/23/06
3                  10/10/00
4                  09/06/06
                ...        
52473         May 28th 2011
52474       August 5th 2011
52475       March 18th 2011
52476    September 1st 2011
52477          May 8th 2011
Name: publishDate, Length: 52478, dtype: object>

In [ ]:
df.shape

(52478, 25)

In [23]:
input_file = "../data/raw/books_1.Best_Books_Ever.csv"
batch_size = 10000
wait_seconds = 1 

def get_google_books_data(title, author=None, isbn=None):
    """
    Fetch book data from Google Books API using title, author, or ISBN.
    """
    try:
        if isbn and isbn != '9999999999999':
            query = f'isbn:{isbn}'
        else:
            query = f'intitle:{title}'
            if author:
                query += f'+inauthor:{author}'
        
        url = f'https://www.googleapis.com/books/v1/volumes?q={query}'
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if 'items' not in data:
            return None
        
        volume = data['items'][0]['volumeInfo']
        return {
            "isbn": next((id["identifier"] for id in volume.get("industryIdentifiers", []) if id["type"] == "ISBN_13"), None),
            "publisher": volume.get("publisher"),
            "publishDate": volume.get("publishedDate"),
            "pages": volume.get("pageCount"),
            "language": volume.get("language"),
            "description": volume.get("description"),
            "coverImg": volume.get("imageLinks", {}).get("thumbnail")
        }
    except RequestException as e:
        print(f"Error in '{title}': {e}")
        return None

def update_missing_fields(row):
    """
    Update missing fields in a row with data from Google Books API.
    """
    needs_update = (
        row['isbn'] == '9999999999999' or
        pd.isna(row['publisher']) or
        pd.isna(row['publishDate']) or
        pd.isna(row['pages']) or
        pd.isna(row['description']) or
        pd.isna(row['coverImg']) or
        pd.isna(row['language'])
    )
    if not needs_update:
        return row
    
    title = row['title']
    author = row['author'].split(',')[0] if pd.notna(row['author']) else None
    isbn = row['isbn'] if row['isbn'] != '9999999999999' else None

    data = get_google_books_data(title, author, isbn)
    time.sleep(wait_seconds)

    if data:
        for key, value in data.items():
            if pd.isna(row.get(key)) or (key == 'isbn' and row['isbn'] == '9999999999999'):
                row[key] = value
    return row

df = pd.read_csv(input_file)
total = len(df)
num_batches = (total // batch_size) + 1

for i in range(num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, total)
    df_batch = df.iloc[start:end].copy()

    print(f"\nProcessing batch {i+1}/{num_batches} — books {start} to {end}")

    df_batch_updated = df_batch.apply(update_missing_fields, axis=1)

    output_name = f"books_enriched_batch_{i+1}.csv"
    df_batch_updated.to_csv(output_name, index=False)
    print(f"Saved: {output_name}")


Processing batch 1/6 — books 0 to 10000
Saved: books_enriched_batch_1.csv

Processing batch 2/6 — books 10000 to 20000
Error in 'عرسنا في الجنة': 503 Server Error: Service Unavailable for url: https://www.googleapis.com/books/v1/volumes?q=intitle:%D8%B9%D8%B1%D8%B3%D9%86%D8%A7%20%D9%81%D9%8A%20%D8%A7%D9%84%D8%AC%D9%86%D8%A9+inauthor:%D8%B9%D8%B2%D8%A7%D9%85%20%D8%AD%D8%AF%D8%A8%D8%A7
Saved: books_enriched_batch_2.csv

Processing batch 3/6 — books 20000 to 30000
Saved: books_enriched_batch_3.csv

Processing batch 4/6 — books 30000 to 40000
Saved: books_enriched_batch_4.csv

Processing batch 5/6 — books 40000 to 50000
Saved: books_enriched_batch_5.csv

Processing batch 6/6 — books 50000 to 52478
Saved: books_enriched_batch_6.csv


In [26]:
input_file = "../data/raw/books_1.Best_Books_Ever.csv" 
batch_size = 10000                        
wait_seconds = 1                   

# Function to query Google Books API
def get_google_books_data(title, author=None, isbn=None):
    try:
        if isbn and isbn != '9999999999999':
            query = f'isbn:{isbn}'
        else:
            query = f'intitle:{title}'
            if author:
                query += f'+inauthor:{author}'
        
        url = f'https://www.googleapis.com/books/v1/volumes?q={query}'
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if 'items' not in data:
            return None
        
        volume = data['items'][0]['volumeInfo']
        return {
            "isbn": next((id["identifier"] for id in volume.get("industryIdentifiers", []) if id["type"] == "ISBN_13"), None),
            "publisher": volume.get("publisher"),
            "publishDate": volume.get("publishedDate"),
            "pages": volume.get("pageCount"),
            "language": volume.get("language"),
            "description": volume.get("description"),
            "coverImg": volume.get("imageLinks", {}).get("thumbnail")
        }
    except RequestException as e:
        print(f"Error in '{title}': {e}")
        return None

# Function to update missing fields
def update_missing_fields(row):
    needs_update = (
        row['isbn'] == '9999999999999' or
        pd.isna(row['publisher']) or
        pd.isna(row['publishDate']) or
        pd.isna(row['pages']) or
        pd.isna(row['description']) or
        pd.isna(row['coverImg']) or
        pd.isna(row['language'])
    )
    if not needs_update:
        return row
    
    title = row['title']
    author = row['author'].split(',')[0] if pd.notna(row['author']) else None
    isbn = row['isbn'] if row['isbn'] != '9999999999999' else None

    data = get_google_books_data(title, author, isbn)
    time.sleep(wait_seconds)

    if data:
        for key, value in data.items():
            if pd.isna(row.get(key)) or (key == 'isbn' and row['isbn'] == '9999999999999'):
                row[key] = value
    return row

print("Loading CSV file...")
df = pd.read_csv(input_file)
total = len(df)
num_batches = (total // batch_size) + 1

processed = [f"books_enriched_batch_{i+1}.csv" for i in range(num_batches)]
start_batch = next((i for i, name in enumerate(processed) if not os.path.exists(name)), num_batches)

if start_batch >= num_batches:
    print("All batches have already been processed.")
else:
    print(f"Starting from batch {start_batch + 1}/{num_batches}...")

for i in range(start_batch, num_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, total)
    df_batch = df.iloc[start:end].copy()

    print(f"\nProcessing batch {i+1}/{num_batches} — books {start} to {end}")

    df_batch_updated = df_batch.apply(update_missing_fields, axis=1)

    output_name = f"books_enriched_batch_{i+1}.csv"
    df_batch_updated.to_csv(output_name, index=False)
    print(f"Saved: {output_name}")

print("\nProcess completed.")


Loading CSV file...
All batches have already been processed.

Process completed.


In [27]:
input_file      = "../data/raw/books_1.Best_Books_Ever.csv"
batch_size      = 10_000
wait_seconds    = 1

start_row = 40_000      

# Function to query Google Books API
def get_google_books_data(title, author=None, isbn=None):
    try:
        if isbn and isbn != '9999999999999':
            query = f'isbn:{isbn}'
        else:
            query = f'intitle:{title}'
            if author:
                query += f'+inauthor:{author}'
        url = f'https://www.googleapis.com/books/v1/volumes?q={query}'
        r = requests.get(url, timeout=10)
        r.raise_for_status()
        data = r.json()
        if 'items' not in data:
            return None
        v = data['items'][0]['volumeInfo']
        return {
            "isbn": next((i["identifier"] for i in v.get("industryIdentifiers", [])
                          if i["type"] == "ISBN_13"), None),
            "publisher":    v.get("publisher"),
            "publishDate":  v.get("publishedDate"),
            "pages":        v.get("pageCount"),
            "language":     v.get("language"),
            "description":  v.get("description"),
            "coverImg":     v.get("imageLinks", {}).get("thumbnail")
        }
    except RequestException as e:
        print(f"Error in '{title}': {e}")
        return None

# Function to update missing fields
def update_missing_fields(row):
    needs = (
        row['isbn'] == '9999999999999' or
        pd.isna(row['publisher']) or pd.isna(row['publishDate']) or
        pd.isna(row['pages']) or pd.isna(row['description']) or
        pd.isna(row['coverImg']) or pd.isna(row['language'])
    )
    if not needs:
        return row
    title  = row['title']
    author = row['author'].split(',')[0] if pd.notna(row['author']) else None
    isbn   = row['isbn'] if row['isbn'] != '9999999999999' else None
    data   = get_google_books_data(title, author, isbn)
    time.sleep(wait_seconds)
    if data:
        for k, v in data.items():
            if pd.isna(row.get(k)) or (k == 'isbn' and row['isbn'] == '9999999999999'):
                row[k] = v
    return row

print("Loading CSV…")
df = pd.read_csv(input_file)
total      = len(df)
num_batches = (total // batch_size) + 1

start_batch = start_row // batch_size
print(f"Processing from batch {start_batch + 1} "
      f"(row {start_row:,} of {total:,}).")

for i in range(start_batch, num_batches):
    start = i * batch_size
    end   = min((i + 1) * batch_size, total)
    if end <= start_row:      
        continue

    df_batch = df.iloc[start:end].copy()
    print(f"\nBatch {i+1}/{num_batches} — rows {start:,} → {end-1:,}")

    df_batch_updated = df_batch.apply(update_missing_fields, axis=1)

    out = f"books_enriched_batch_{i+1}.csv"
    df_batch_updated.to_csv(out, index=False)
    print(f"Saved: {out}")

print("\nProcess completed.")


Loading CSV…
Processing from batch 5 (row 40,000 of 52,478).

Batch 5/6 — rows 40,000 → 49,999
Saved: books_enriched_batch_5.csv

Batch 6/6 — rows 50,000 → 52,477
Saved: books_enriched_batch_6.csv

Process completed.


In [28]:
# ========== CONFIGURATION ==========
input_file   = "../data/raw/books_1.Best_Books_Ever.csv"
output_file  = "books_enriched_30000_40000.csv"
start_row    = 30000
end_row      = 40000
wait_seconds = 1
# ===================================

# Function to query Google Books API
def get_google_books_data(title, author=None, isbn=None):
    try:
        if isbn and isbn != '9999999999999':
            query = f'isbn:{isbn}'
        else:
            query = f'intitle:{title}'
            if author:
                query += f'+inauthor:{author}'
        url = f'https://www.googleapis.com/books/v1/volumes?q={query}'
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        if 'items' not in data:
            return None
        v = data['items'][0]['volumeInfo']
        return {
            "isbn": next((i["identifier"] for i in v.get("industryIdentifiers", [])
                          if i["type"] == "ISBN_13"), None),
            "publisher":    v.get("publisher"),
            "publishDate":  v.get("publishedDate"),
            "pages":        v.get("pageCount"),
            "language":     v.get("language"),
            "description":  v.get("description"),
            "coverImg":     v.get("imageLinks", {}).get("thumbnail")
        }
    except RequestException as e:
        print(f"⚠️ Error in '{title}': {e}")
        return None

# Function to update missing fields
def update_missing_fields(row):
    needs = (
        row['isbn'] == '9999999999999' or
        pd.isna(row['publisher']) or pd.isna(row['publishDate']) or
        pd.isna(row['pages']) or pd.isna(row['description']) or
        pd.isna(row['coverImg']) or pd.isna(row['language'])
    )
    if not needs:
        return row
    title  = row['title']
    author = row['author'].split(',')[0] if pd.notna(row['author']) else None
    isbn   = row['isbn'] if row['isbn'] != '9999999999999' else None
    data   = get_google_books_data(title, author, isbn)
    time.sleep(wait_seconds)
    if data:
        for k, v in data.items():
            if pd.isna(row.get(k)) or (k == 'isbn' and row['isbn'] == '9999999999999'):
                row[k] = v
    return row

print(f"Loading CSV from row {start_row:,} to {end_row-1:,}…")
df = pd.read_csv(input_file)

# Extract only rows 30,000 to 39,999
df_batch = df.iloc[start_row:end_row].copy()
print(f"Processing batch of {len(df_batch):,} books…")

df_batch_updated = df_batch.apply(update_missing_fields, axis=1)

# Save result
df_batch_updated.to_csv(output_file, index=False)
print(f"Saved: {output_file}")


Loading CSV from row 30,000 to 39,999…
Processing batch of 10,000 books…
⚠️ Error in 'The Boy Lost In A World': 503 Server Error: Service Unavailable for url: https://www.googleapis.com/books/v1/volumes?q=isbn:B07Q5VDN8G
Saved: books_enriched_30000_40000.csv


In [29]:
# List of files to merge 
files = [
    "../data/interim/books_enriched_batch_1.csv",
    "../data/interim/books_enriched_batch_2.csv",
    "../data/interim/books_enriched_batch_3.csv",
    "../data/interim/books_enriched_batch_4.csv",
    "../data/interim/books_enriched_30000_40000.csv",
    "../data/interim/books_enriched_batch_5.csv",
    "../data/interim/books_enriched_batch_6.csv",
]

# Read and concatenate all
dataframes = [pd.read_csv(file) for file in files]
df_final = pd.concat(dataframes, ignore_index=True)

# Save to a new file
df_final.to_csv("../data/processed/books_enriched_full.csv", index=False)
print("Combined file saved as 'books_enriched_full.csv'")

Combined file saved as 'books_enriched_full.csv'


In [31]:
df_full = pd.read_csv('../data/processed/books_enriched_full.csv')
df_full

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaN,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,06/21/03,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9780329001186,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,07/11/60,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
3,1885.Pride_and_Prejudice,Pride and Prejudice,NaN,"Jane Austen, Anna Quindlen (Introduction)",4.26,Alternate cover edition of ISBN 9780679783268S...,English,NaN,"['Classics', 'Fiction', 'Romance', 'Historical...","['Mr. Bennet', 'Mrs. Bennet', 'Jane Bennet', '...",...,01/28/13,[],2998241,"['1617567', '816659', '373311', '113934', '767...",94.0,"['United Kingdom', 'Derbyshire, England (Unite...",https://i.gr-assets.com/images/S/compressed.ph...,1983116,20452,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,10/05/05,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62473,11492014-fractured,Fractured,Fateful #2,Cheri Schmidt (Goodreads Author),4.00,The Fateful Trilogy continues with Fractured. ...,English,2940012616562,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",[],...,NaN,[],871,"['311', '310', '197', '42', '11']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN
62474,11836711-anasazi,Anasazi,Sense of Truth #2,Emma Michaels,4.19,"'Anasazi', sequel to 'The Thirteenth Chime' by...",English,9780615510545,"['Mystery', 'Young Adult']",[],...,August 3rd 2011,[],37,"['16', '14', '5', '2', '0']",95.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN
62475,10815662-marked,Marked,Soul Guardians #1,Kim Richardson (Goodreads Author),3.70,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,9781461017097,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange...",[],...,March 15th 2011,"[""Readers' Favorite Book Award (2011)""]",6674,"['2109', '1868', '1660', '647', '390']",84.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,7.37
62476,11330278-wayward-son,Wayward Son,NaN,"Tom Pollack (Goodreads Author), John Loftus (G...",3.85,A POWERFUL TREMOR UNEARTHS AN ANCIENT SECRETBu...,English,978145075

In [32]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62478 entries, 0 to 62477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            62478 non-null  object 
 1   title             62478 non-null  object 
 2   series            27701 non-null  object 
 3   author            62478 non-null  object 
 4   rating            62478 non-null  float64
 5   description       61364 non-null  object 
 6   language          60979 non-null  object 
 7   isbn              61448 non-null  object 
 8   genres            62478 non-null  object 
 9   characters        62478 non-null  object 
 10  bookFormat        60621 non-null  object 
 11  edition           5750 non-null   object 
 12  pages             60857 non-null  object 
 13  publisher         58812 non-null  object 
 14  publishDate       62004 non-null  object 
 15  firstPublishDate  36819 non-null  object 
 16  awards            62478 non-null  object

In [33]:
df_full['publishDate'].value_counts()

publishDate
Published             757
2008                  327
2007                  288
2009                  287
2010                  264
                     ... 
November 17th 2006      1
July 26th 2000          1
August 30th 2010        1
02/28/99                1
September 3rd 2000      1
Name: count, Length: 10838, dtype: int64

In [34]:
df_full['firstPublishDate'].value_counts()

firstPublishDate
01/01/12               290
01/01/10               267
01/01/11               259
01/01/08               230
01/01/13               225
                      ... 
May 15th 1999            1
January 17th 2008        1
October 18th 1990        1
September 12th 2004      1
December 1st 1979        1
Name: count, Length: 8024, dtype: int64

In [35]:
before = len(df_full)
df_full = df_full[df_full['isbn'].notna() & (df_full['isbn'].str.strip() != '')]
after = len(df_full)
print(f" {before - after} rows without ISBN removed.")

 1030 rows without ISBN removed.


In [36]:
def safe_parse_date(date):
    try:
        dt = parser.parse(str(date), fuzzy=True)
        if 1700 <= dt.year <= 2100:
            return dt  
        else:
            return pd.NaT
    except:
        return pd.NaT
    
df_full.loc[:, 'publishDate'] = df_full['publishDate'].apply(safe_parse_date)
df_full.loc[:, 'firstPublishDate'] = df_full['firstPublishDate'].apply(safe_parse_date)
print(df_full[['publishDate', 'firstPublishDate']].head())


           publishDate     firstPublishDate
0  2008-09-14 00:00:00                  NaT
1  2004-09-28 00:00:00  2003-06-21 00:00:00
2  2006-05-23 00:00:00  2060-07-11 00:00:00
4  2006-09-06 00:00:00  2005-10-05 00:00:00
5  2006-03-14 00:00:00  2005-09-01 00:00:00


In [37]:
df_full

,bookId,title,series,author,rating,description,language,isbn,genres,characters,...,firstPublishDate,awards,numRatings,ratingsByStars,likedPercent,setting,coverImg,bbeScore,bbeVotes,price
0,2767052-the-hunger-games,The Hunger Games,The Hunger Games #1,Suzanne Collins,4.33,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,English,9780439023481,"['Young Adult', 'Fiction', 'Dystopia', 'Fantas...","['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...",...,NaT,['Locus Award Nominee for Best Young Adult Boo...,6376780,"['3444695', '1921313', '745221', '171994', '93...",96.0,"['District 12, Panem', 'Capitol, Panem', 'Pane...",https://i.gr-assets.com/images/S/compressed.ph...,2993816,30516,5.09
1,2.Harry_Potter_and_the_Order_of_the_Phoenix,Harry Potter and the Order of the Phoenix,Harry Potter #5,"J.K. Rowling, Mary GrandPré (Illustrator)",4.50,There is a door at the end of a silent corrido...,English,9780439358071,"['Fantasy', 'Young Adult', 'Fiction', 'Magic',...","['Sirius Black', 'Draco Malfoy', 'Ron Weasley'...",...,2003-06-21 00:00:00,['Bram Stoker Award for Works for Young Reader...,2507623,"['1593642', '637516', '222366', '39573', '14526']",98.0,['Hogwarts School of Witchcraft and Wizardry (...,https://i.gr-assets.com/images/S/compressed.ph...,2632233,26923,7.38
2,2657.To_Kill_a_Mockingbird,To Kill a Mockingbird,To Kill a Mockingbird,Harper Lee,4.28,The unforgettable novel of a childhood in a sl...,English,9780329001186,"['Classics', 'Fiction', 'Historical Fiction', ...","['Scout Finch', 'Atticus Finch', 'Jem Finch', ...",...,2060-07-11 00:00:00,"['Pulitzer Prize for Fiction (1961)', 'Audie A...",4501075,"['2363896', '1333153', '573280', '149952', '80...",95.0,"['Maycomb, Alabama (United States)']",https://i.gr-assets.com/images/S/compressed.ph...,2269402,23328,NaN
4,41865.Twilight,Twilight,The Twilight Saga #1,Stephenie Meyer,3.60,About three things I was absolutely positive.\...,English,9780316015844,"['Young Adult', 'Fantasy', 'Romance', 'Vampire...","['Edward Cullen', 'Jacob Black', 'Laurent', 'R...",...,2005-10-05 00:00:00,"['Georgia Peach Book Award (2007)', 'Buxtehude...",4964519,"['1751460', '1113682', '1008686', '542017', '5...",78.0,"['Forks, Washington (United States)', 'Phoenix...",https://i.gr-assets.com/images/S/compressed.ph...,1459448,14874,2.1
5,19063.The_Book_Thief,The Book Thief,NaN,Markus Zusak (Goodreads Author),4.37,Librarian's note: An alternate cover edition c...,English,9780375831003,"['Historical Fiction', 'Fiction', 'Young Adult...","['Liesel Meminger', 'Hans Hubermann', 'Rudy St...",...,2005-09-01 00:00:00,['National Jewish Book Award for Children’s an...,1834276,"['1048230', '524674', '186297', '48864', '26211']",96.0,"['Molching (Germany)', 'Germany']",https://i.gr-assets.com/images/S/compressed.ph...,1372809,14168,3.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62473,11492014-fractured,Fractured,Fateful #2,Cheri Schmidt (Goodreads Author),4.00,The Fateful Trilogy continues with Fractured. ...,English,2940012616562,"['Vampires', 'Paranormal', 'Young Adult', 'Rom...",[],...,NaT,[],871,"['311', '310', '197', '42', '11']",94.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN
62474,11836711-anasazi,Anasazi,Sense of Truth #2,Emma Michaels,4.19,"'Anasazi', sequel to 'The Thirteenth Chime' by...",English,9780615510545,"['Mystery', 'Young Adult']",[],...,2011-08-03 00:00:00,[],37,"['16', '14', '5', '2', '0']",95.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,NaN
62475,10815662-marked,Marked,Soul Guardians #1,Kim Richardson (Goodreads Author),3.70,--READERS FAVORITE AWARDS WINNER 2011--Sixteen...,English,9781461017097,"['Fantasy', 'Young Adult', 'Paranormal', 'Ange...",[],...,2011-03-15 00:00:00,"[""Readers' Favorite Book Award (2011)""]",6674,"['2109', '1868', '1660', '647', '390']",84.0,[],https://i.gr-assets.com/images/S/compressed.ph...,0,1,7.37
62476,11330278-wayward-son,Wayward Son,NaN,"Tom Pollack (Goodreads Author), John Loftus (G...",

In [38]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61448 entries, 0 to 62477
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   bookId            61448 non-null  object 
 1   title             61448 non-null  object 
 2   series            27468 non-null  object 
 3   author            61448 non-null  object 
 4   rating            61448 non-null  float64
 5   description       60489 non-null  object 
 6   language          59949 non-null  object 
 7   isbn              61448 non-null  object 
 8   genres            61448 non-null  object 
 9   characters        61448 non-null  object 
 10  bookFormat        59819 non-null  object 
 11  edition           5564 non-null   object 
 12  pages             59855 non-null  object 
 13  publisher         57925 non-null  object 
 14  publishDate       60002 non-null  object 
 15  firstPublishDate  36263 non-null  object 
 16  awards            61448 non-null  object 
 17

In [39]:
# Check columns by position (20 to 22 inclusive)
df_full.iloc[:, 18:25].isna().sum()

ratingsByStars        0
likedPercent        884
setting               0
coverImg            459
bbeScore              0
bbeVotes              0
price             16559
dtype: int64

In [40]:
# See how many rows have at least one missing value in key columns
target_columns = ['description', 'language', 'bookFormat', 'pages', 'publishDate', 'coverImg']
incomplete_rows = df_full[target_columns].isna().any(axis=1)
total_incomplete = incomplete_rows.sum()
print(f"There are {total_incomplete} rows with at least one missing value in the selected columns.")

There are 5265 rows with at least one missing value in the selected columns.


In [41]:
# Remove rows with missing key data
df_full = df_full[~incomplete_rows].copy()


In [42]:
print(f"Number of rows after removing the incomplete ones: {len(df_full)}")

Number of rows after removing the incomplete ones: 56183


In [43]:
print(df_full.iloc[0])

bookId                                       2767052-the-hunger-games
title                                                The Hunger Games
series                                            The Hunger Games #1
author                                                Suzanne Collins
rating                                                           4.33
description         WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...
language                                                      English
isbn                                                    9780439023481
genres              ['Young Adult', 'Fiction', 'Dystopia', 'Fantas...
characters          ['Katniss Everdeen', 'Peeta Mellark', 'Cato (H...
bookFormat                                                  Hardcover
edition                                                 First Edition
pages                                                             374
publisher                                            Scholastic Press
publishDate         

In [44]:
print(df_full.iloc[0]['setting'])

['District 12, Panem', 'Capitol, Panem', 'Panem (United States)']
